In [456]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [457]:
df_gdp = pd.read_excel("india_gdp.xls")
df_gdp.head()

,observation_date,INDLORSGPNOSTSAM
0,1996-05-01,100.911191
1,1996-06-01,100.817508
2,1996-07-01,100.724427
3,1996-08-01,100.632440
4,1996-09-01,100.541580


In [458]:
df_gpr = pd.read_excel("india_gpr.xls")
df_gpr.head()

,month,GPRHC_IND
0,1900-01-01,0.271880
1,1900-02-01,0.390353
2,1900-03-01,0.283432
3,1900-04-01,0.120192
4,1900-05-01,0.217391


In [459]:
df_tmp = pd.read_excel("india_temperature.xls")
df_tmp.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1901,22.40,24.14,29.07,31.91,33.41,33.18,31.21,30.39,30.47,29.97,27.31,24.49
1,1902,24.93,26.58,29.77,31.78,33.73,32.91,30.92,30.73,29.80,29.12,26.31,24.04
2,1903,23.44,25.03,27.83,31.39,32.91,33.00,31.34,29.98,29.85,29.04,26.08,23.65
3,1904,22.50,24.73,28.21,32.02,32.64,32.07,30.36,30.09,30.04,29.20,26.36,23.63
4,1905,22.00,22.83,26.68,30.01,33.32,33.25,31.44,30.68,30.12,30.67,27.52,23.82


In [460]:
df_gold = pd.read_csv("india_gold_price.csv")
df_gold.head()

,Date,India(INR)
0,31-01-1979,1840.8
1,28-02-1979,2011.7
2,30-03-1979,1940.2
3,30-04-1979,2013.1
4,31-05-1979,2253.4


In [461]:
# Reshape the DataFrame
df_tmp = df_tmp.melt(id_vars=['YEAR'], var_name='MONTH', value_name='VALUE')

# Sort the DataFrame by year
df_tmp.sort_values(by=['YEAR', 'MONTH'], inplace=True)

# Reset index
df_tmp.reset_index(drop=True, inplace=True)

In [462]:
df_tmp.head()

,YEAR,MONTH,VALUE
0,1901,APR,31.91
1,1901,AUG,30.39
2,1901,DEC,24.49
3,1901,FEB,24.14
4,1901,JAN,22.40


In [463]:
month_to_number = {
    'JAN': '01',
    'FEB': '02',
    'MAR': '03',
    'APR': '04',
    'MAY': '05',
    'JUN': '06',
    'JUL': '07',
    'AUG': '08',
    'SEP': '09',
    'OCT': '10',
    'NOV': '11',
    'DEC': '12'
}
df_tmp['Date'] = '01'
df_tmp['MONTH'] = df_tmp['MONTH'].map(month_to_number)
# Concatenate year, month, and date to create the final date column
df_tmp['Date'] =  df_tmp['Date'] + '-' + df_tmp['MONTH'] + '-' + df_tmp['YEAR'].astype(str)

# Drop unnecessary columns
df_tmp.drop(columns=['YEAR', 'MONTH'], inplace=True)
df_tmp = df_tmp.loc[:,['Date', 'VALUE']]

In [464]:
df_tmp.head()

,Date,VALUE
0,01-04-1901,31.91
1,01-08-1901,30.39
2,01-12-1901,24.49
3,01-02-1901,24.14
4,01-01-1901,22.40


In [465]:
df_tmp['Date'] = pd.to_datetime(df_tmp['Date'], format='%d-%m-%Y')
df_tmp['Date'] = df_tmp['Date'].dt.to_period('M')

In [466]:
df_gdp.rename(columns={'observation_date': 'Date', 'INDLORSGPNOSTSAM':'GDP'}, inplace=True)
df_gdp['Date'] = pd.to_datetime(df_gdp['Date'], format='%Y-%m-%d')
df_gdp['Date'] = df_gdp['Date'].dt.to_period('M')

In [467]:
df_gpr.rename(columns={'month': 'Date'}, inplace=True)
df_gpr['Date'] = pd.to_datetime(df_gpr['Date'], format='%Y-%m-%d')
df_gpr['Date'] = df_gpr['Date'].dt.to_period('M')

In [468]:
df_gold.rename(columns={'India(INR)': 'Gold_Price'}, inplace=True)
df_gold['Date'] = pd.to_datetime(df_gold['Date'], format='%d-%m-%Y')
df_gold['Date'] = df_gold['Date'].dt.to_period('M')

In [469]:
df_merged = df_tmp.merge(df_gdp, on='Date', how='inner').merge(df_gpr, on='Date', how='inner').merge(df_gold, on='Date', how='inner')

In [470]:
df = df_merged.copy()

In [471]:
df.head()

,Date,VALUE,GDP,GPRHC_IND,Gold_Price
0,1996-08,30.23,100.632440,0.226945,13784.7
1,1996-12,24.83,100.266565,0.117710,13228.4
2,1996-07,31.19,100.724427,0.190993,13697.4
3,1996-06,32.50,100.817508,0.263451,13457.9
4,1996-05,33.69,100.911191,0.145405,13681.0


In [472]:
df.shape

(284, 5)

In [473]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype    
---  ------      --------------  -----    
 0   Date        284 non-null    period[M]
 1   VALUE       284 non-null    float64  
 2   GDP         284 non-null    float64  
 3   GPRHC_IND   284 non-null    float64  
 4   Gold_Price  284 non-null    float64  
dtypes: float64(4), period[M](1)
memory usage: 13.3 KB


In [474]:
df['Date'] = pd.to_datetime(df['Date'].astype(str) + '-01')
df = df.sort_values(by='Date')

In [475]:
df.head()

,Date,VALUE,GDP,GPRHC_IND,Gold_Price
4,1996-05-01,33.69,100.911191,0.145405,13681.0
3,1996-06-01,32.50,100.817508,0.263451,13457.9
2,1996-07-01,31.19,100.724427,0.190993,13697.4
0,1996-08-01,30.23,100.632440,0.226945,13784.7
7,1996-09-01,30.86,100.541580,0.203874,13511.3


In [476]:
df.to_csv("gold_price_prediction.csv",index=False)